
# Advisory mission for the Boston municipal authorities


![title](assets/boston_logo.png)

**Welcome to the Decision Science challenge of your data certification exam!**

⚠️ This challenge validates 2 blocks of the RNCP certification _Développeur en intelligence artificielle_ exam:  

**Block 1:**  *Analyse exploratoire et descriptive de la donnée*  
- 75% validated by Section 1 - Analysis of the mayor's team
- 25% validated by Section 2 - Analysis for the police department

**Block 2:**  *Développement et Gestion de projet en IA*  
- 100% validated by Section 2 - Analysis for the police department

To start, here is an overview of the `decision_science` directory:

In [165]:
# Start by running this cell to see the architecture of the directory
!tree

.
├── assets
│   └── boston_logo.png
├── boston_crimes.ipynb
├── boston_plots_for_slides.ipynb
├── CRIME_ANALYSIS_Boston_Police_Department.pdf
├── data
│   ├── boston_crimes_regression.csv
│   ├── districts.csv
│   └── incident_reports.csv
├── data.py
├── db
│   ├── boston_crimes.sqlite
│   └── boston_crimes.xml
├── Makefile
├── __pycache__
│   └── data.cpython-310.pyc
├── README.md
└── tests
    ├── answers_inference.json
    ├── linear_regression_analysis.pickle
    ├── linear_regression_model.pickle
    ├── manipulation.pickle
    └── sql.pickle

5 directories, 18 files


- the `boston_crimes.ipynb` notebook that you currently have under your eyes is the main document. You will find all the instructions here and except when it is explicitly specified, you should provide all your answers in this notebook;


- the `data` and `db` folders will be filled-in throughout the exercise respectively with `.csv` datasets and a `.sqlite` file, for you to run all your analyses; 


- you will not have to interact with the `assets` folder for this exercise;


- the `tests` folder will contain all the `.pickle` files that will be saved throughout the exercise with your key findings. Please run all the "Save your results" cells when completing the exercise!

⚠️ **Important remark** before you dive into the exercise. This notebook is quite long and it is easy to get lost in it: take full advantage of the collapsible headers and of the table of content. If you have not yet activated these Jupyter Notebook extensions, you may consider doing so now!

# Imports

You can use this section to run your imports in a centralised manner throughout the exercise.

In [2]:
# Load the nbresult package to be able to save your results 
from nbresult import ChallengeResult

In [3]:
# Useful import for data collection
import sqlite3

In [4]:
# Useful imports for data manipulation and analysis
import numpy as np
import pandas as pd

In [5]:
# Useful imports for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# Useful imports to estimate regression models
import statsmodels.formula.api as smf

# 1. Analysis for the mayor's team (4h)

**This part assesse the following skills of the first block of the certification _Analyse exploratoire et descriptive de la donnée_**:
- C1 Acquérir des données à partir de sources adaptées, fournies par le client ou accessibles en Open Data afin d'agrémenter la base de données
- C2 Écrire des requêtes pour l'interrogation de la base de données et les composants d'accès aux données
- C3 Effectuer des analyses exploratoires afin d'appréhender la composition interne des données

During the last municipal campaign in Boston, criminality has been a major topic of debates. As citizens have expressed strong expectations from her on that front, the newly-elected mayor of Boston is looking for data-based insights on criminality in the Massachussetts capital. She has mandated your economics and urbanism consulting firm, *The Locomotive*, for this study.

## 1.1 Load the database

_C1 Acquérir des données à partir de sources adaptées, fournies par le client ou accessibles en Open Data afin d'agrémenter la base de données_

Download the `boston_crimes.sqlite` database from this [URL](https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite) and store it inside the `db` folder.

In [7]:
# You may directly run this cell to do so
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite > db/boston_crimes.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.1M  100 19.1M    0     0  3112k      0  0:00:06  0:00:06 --:--:-- 2632k


## 1.2 Familiarize yourself with the database (40 min)

This section aims at giving you a first overview of the database. 

As you will see, it consists in three tables: 

- the `incident_reports` table has been provided by the Police Department of Boston. Each observation corresponds to a criminal incident that has required an intervention by the police in the municipality of Boston;  



- the `districts` table has been provided by the Urbanism Department of Boston. It gathers geographical information about the various police districts of Boston;  



- and the `indicators` table has been shared by the Economics Department of Boston, which keeps track of various indicators of the social and economic activity of Boston neighborhoods. Each observation corresponds to a police district.

More information about the different fields of these three tables can be found in the dedicated `README.md` file

### Connect to the database

**🔍 Using your preferred SQL client, connect to the database and browse through it as you wish to get acquainted with the data.**

### Draw a schema of the database

**📝 Draw the database schema thanks to the [schema editor](https://kitt.lewagon.com/db) on Kitt.**

**📝 Download the schema and save it as `boston_crimes.xml` in the `db` folder.**

## 1.3 Extract the relevant dataset (40 min)

_C2 Écrire des requêtes pour l'interrogation de la base de données et les composants d'accès aux données_

Now that you have a good overview of the database, you can kick off the work! You will start with an SQL query to gather the relevant information.

### Build the dataset

We want to investigate the influence of the socio-economic characteristics of Boston's different districts on the number of crime reports and incidents. To do so, we need to extract the relevant dataset. **Each row should correspond to one of the 12 police districts of Boston** (as listed in the `districts` table of the database).

To identify the district, we will need **the following columns**: 

- the `CODE` of the police district (1 letter and 1 or 2 numbers);
- the full `NAME` of the police district.

Additionally, you will need to **create an additional field** (which will serve as dependent variable in future regressions): `NB_INCIDENTS`, i.e. the total number of incidents reported in the police district over the period covered by the data at hand (2015-2019).

Eventually, we want the dataset to **include several socio-economic indicators**:

- `MEDIAN_AGE`;
- `TOTAL_POP`;
- `PERC_OF_30_34`;
- `PERC_MARRIED_COUPLE_FAMILY`;
- `PER_CAPITA_INCOME`;
- `PERC_OTHER_STATE_OR_ABROAD`;
- `PERC_LESS_THAN_HIGH_SCHOOL`;
- `PERC_COLLEGE_GRADUATES`. 

Overall, your dataset should comprise 12 rows and 11 columns.

Eventually, note that **the resulting DataFrame must be ordered by the number of incidents**, from the largest to the smallest total.

**📝 Write the SQL query you need to fetch the data. Save it as a `str` in the `query` variable.**

In [8]:
query = """
SELECT d.CODE,
    d.NAME,
    COUNT(ir.INCIDENT_NUMBER) NB_INCIDENTS,
    i.MEDIAN_AGE,
    i.TOTAL_POP,
    i.PERC_OF_30_34,
    i.PERC_MARRIED_COUPLE_FAMILY,
    i.PER_CAPITA_INCOME,
    i.PERC_OTHER_STATE_OR_ABROAD,
    i.PERC_LESS_THAN_HIGH_SCHOOL,
    i.PERC_COLLEGE_GRADUATES 
FROM incident_reports ir 
JOIN districts d ON ir.DISTRICT = d.CODE 
JOIN indicators i ON i.CODE = d.CODE
GROUP BY d.CODE, d.NAME  
ORDER BY NB_INCIDENTS DESC 

"""

**📝 Store the output of the query in a `DataFrame` named `crimes_df`. Display the 5 first rows, as well as the shape of the dataset.**

In [9]:
conn = sqlite3.connect("db/boston_crimes.sqlite")

crimes_df = pd.read_sql_query(query, conn)
crimes_df

,CODE,NAME,NB_INCIDENTS,MEDIAN_AGE,TOTAL_POP,PERC_OF_30_34,PERC_MARRIED_COUPLE_FAMILY,PER_CAPITA_INCOME,PERC_OTHER_STATE_OR_ABROAD,PERC_LESS_THAN_HIGH_SCHOOL,PERC_COLLEGE_GRADUATES
0,B2,Roxbury,38877,32.5,54161,27.8,17.8,20978,2.9,23.0,18.9
1,C11,Dorchester,32875,33.4,126909,28.2,26.6,29767,2.4,18.0,17.1
2,D4,South End,31258,37.1,32571,33.9,28.3,83609,6.2,11.8,8.5
3,B3,Mattapan,28331,36.7,26659,20.9,29.8,28356,2.3,14.5,22.9
4,A1,Downtown,26260,33.5,18306,32.5,35.8,80057,14.8,15.4,6.9
5,C6,South Boston,16617,31.9,36772,46.1,24.7,64745,2.4,7.9,8.4
6,D14,Brighton,13788,30.8,55297,52.8,26.4,41261,8.6,6.7,10.5
7,E13,Jamaica Plain,12802,34.8,40867,32.5,33.7,51655,5.5,8.0,12.1
8,E18,Hyde Park,12551,39.4,38924,21.1,38.4,32744,1.9,13.8,21.3
9,A7,East Boston,9691,30.6,47263,31.1,30.4,31473,3.5,27.2,11.5


### 💾 Run the cell below to submit your results

In [10]:
from nbresult import ChallengeResult

result = ChallengeResult('sql', query=query, data=crimes_df)
result.write()

## 1.4 Linear regression - The socio-economic determinants of criminality (2h40)

As mentioned above, we want to investigate the impact of the socio-economic characteristics of the different Boston police districts on the number of incidents that are reported in these areas. 
- We are going to use the number of incidents as dependent variable 
- our regressors will be the various socio-economic indicators extracted from the database.

*C3 Effectuer des analyses exploratoires afin d'appréhender la composition interne des données (50%)*

### 1.4.1 Start from a fresh dataset

To make sure that you are using the right data, you can load a fresh dataset from this [URL](https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes_regression.csv).

**📝 Load the data into a DataFrame named `data`**

In [147]:
data = pd.read_csv("data/boston_crimes_regression.csv")
data

,MEDIAN_AGE,TOTAL_POP,PERC_OF_30_34,PERC_MARRIED_COUPLE_FAMILY,PER_CAPITA_INCOME,PERC_OTHER_STATE_OR_ABROAD,PERC_LESS_THAN_HIGH_SCHOOL,PERC_COLLEGE_GRADUATES,CODE,NB_INCIDENTS,NAME
0,30.8,55297,52.8,26.4,41261,8.6,6.7,10.5,D14,13788,Brighton
1,35.7,19890,28.2,36.4,75339,3.4,7.9,8.2,A15,4765,Charlestown
2,33.4,126909,28.2,26.6,29767,2.4,18.0,17.1,C11,32875,Dorchester
3,33.5,18306,32.5,35.8,80057,14.8,15.4,6.9,A1,26260,Downtown
4,30.6,47263,31.1,30.4,31473,3.5,27.2,11.5,A7,9691,East Boston
5,39.4,38924,21.1,38.4,32744,1.9,13.8,21.3,E18,12551,Hyde Park
6,34.8,40867,32.5,33.7,51655,5.5,8.0,12.1,E13,12802,Jamaica Plain
7,36.7,26659,20.9,29.8,28356,2.3,14.5,22.9,B3,28331,Mattapan
8,32.5,54161,27.8,17.8,20978,2.9,23.0,18.9,B2,38877,Roxbury
9,31.9,36772,46.1,24.7,64745,2.4,7.9,8.4,C6,16617,South Boston


### 1.4.2 Run the regression and output its summary

Thanks to the Statsmodels Formula API, we will run the regression described below. 

The dependent variable (or target variable) should be **the total number of incidents** reported in each police district.

We will focus on the following regressors: 

- the **median age** in the district, whose effect is difficult to anticipate on the number of crimes;
 
 
- the **percentage of 30-34 years old** in the district, whose effect is also unclear a priori;
 
 
- the **share of families with a married couple** among all households, which could be anticipated to have a negative effect on criminality (more attention to safety among residents...);
 
 
- the **percentage of residents having moved from abroad or from another US state over the last year**, mobility being often associated with social marginalisation and possibly with a higher risk of resorting to illegal activities;
 
 
- the **percentage of residents having stopped their studies before getting a high school degree**. Economic models would suggest that due to the more narrow job opportunities to which this group has access, the incentive is stronger to resort to illicit activities;
 
 
- the **percentage of college graduates** in the district, which we would expect to have an opposite effect.
 
**📝 Based on these indications, estimate the linear regression model and output its summary in this section of the notebook. Store the estimated model inside a `model` variable.**

In [148]:
model = smf.ols(formula = 'NB_INCIDENTS ~ MEDIAN_AGE + PERC_OF_30_34 + PERC_MARRIED_COUPLE_FAMILY + PERC_OTHER_STATE_OR_ABROAD + PERC_LESS_THAN_HIGH_SCHOOL + PERC_COLLEGE_GRADUATES', data=data).fit()
model.summary()

/home/steph/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           NB_INCIDENTS   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     7.391
Date:                Thu, 22 Dec 2022   Prob (F-statistic):             0.0222
Time:                        15:01:22   Log-Likelihood:                -114.59
No. Observations:                  12   AIC:                             243.2
Df Residuals:                       5   BIC:                             246.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   1.358e+04   6.16e+04      0.220      0.834   -1.45e+05    1.72e+05
MEDIAN_AGE                  2252.7344   1237.522      1.820      0.128    -928.416    5433.885
PERC_OF_30_34               -532.9973    468.940     -1.137      0.307   -1738.447     672.452
PERC_MARRIED_COUPLE_FAMILY -2115.1839    408.344     -5.180      0.004   -3164.866   -1065.502
PERC_OTHER_STATE_OR_ABROAD  1787.6939    532.432      3.358      0.020     419.034    3156.354
PERC_LESS_THAN_HIGH_SCHOOL   112.7375    493.977      0.228      0.829   -1157.072    1382.547
PERC_COLLEGE_GRADUATES        19.4699    458.775      0.042      0.968   -1159.850    1198.790
==============================================================================
Omnibus:                        6.933   Durbin-Watson:                   2.255
Prob(Omnibus):                  0.031   Jarque-Bera (JB):                3.155
Skew:                           1.148   Prob(JB):                        0.206
Kurtosis:                       4.020   Cond. No.                     2.42e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.42e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 💾 Run the cell below to submit your results

In [13]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'linear_regression_model',
    data=data,
    model=model
)
result.write()

### 1.4.3 Statistical inference questions 🤯

*C3 Effectuer des analyses exploratoires afin d'appréhender la composition interne des données (50%)*

You will now be asked a series of statistical inference and methodological questions about the regression model estimated above. Don't worry if results do not perfectly fit the "predictions" that we made earlier about the impact of each regressor: the goal was to form an *a priori* hypothesis, which is validated or invalidated through this empirical analysis.

#### Questions on the results

**❓ Is the median age associated with a positive (increasing the number of crimes) or a negative (decreasing the target variable) effect on the number of crime incidents? Simply write your answer as a string below**

In [149]:
answer_median_age = '''
Le coefficient de MEDIAN_AGE est positif, il influence donc NB_INCIDENTS de manière positive : en ne modifiant pas les autres 
régresseurs, si on modifie l'age médian d'un point, cela augmentera le nombre d'incidents de 2252,7344.
'''

**❓ What is the t-statistic associated with the median age regressor? How is it computed?**

💡 Hint: We are looking at a ratio

In [150]:
answer_t_statistic = '''
La statistique t associée à l'age médian est celle du test de significativité du coefficient associé à l'âge médian.
Elle se calcule par le ratio entre l'estimateur du coefficient et son erreur standard.
'''

**📝 Recompute approximately the t-statistic based on the regression summary.**

As it is a ratio $t = \frac{numerator}{denominator}$:
- Store the numerator into a `numerator` variable
- Store the denominator into a `denominator` variable
- Store the t-statistic into a `t_median_age` variable

In [151]:
numerator = 2252.7344
denominator = 1237.522
t_median_age = numerator / denominator
t_median_age

1.8203590724043692

**❓ What is the p-value associated with the median age regressor? Store it in the `pvalue_median_age` variable (you may directly copy-paste it from the regression summary).** 

In [152]:
pvalue_median_age = 0.128

**❓ What does this p-value mean for the median age? Is its effect statistically significant at the 95% confidence level? At the 90% confidence level? Simply formulate your answer in the cell below.**

In [153]:
answer_p_value = """
La p-value est le degré de significativité pour le test d'hypothèse 'coef. de MEDIAN_AGE = 0', autrement dit, est-ce que 
l'age médian a un effet significatif, ajusté sur les autres var du modèle (30-34, married, other state, high school and 
graduate), sur le nb d'incidents.
Au niveau de confiance 95%, c'est-à-dire un niveau de significativité de 5%, on ne rejette pas l'hypothèse, puisque 0.128 > 0.05
Au niveau de confiance 90%, c'est-à-dire un niveau de significativité de 10%, on ne rejette pas l'hypohèse, puisque 0.128 > 0.10
"""

**❓ What are the two regressors whose effect is statistically significant at the 95% confidence level in this regression model? Store the name of the variables as a list of strings in the `significant_regressors` variable.**

In [154]:
significant_regressors = ["PERC_MARRIED_COUPLE_FAMILY", "PERC_OTHER_STATE_OR_ABROAD"]

#     p-value PERC_MARRIED_COUPLE_FAMILY = 0.004 < 0.05
#     p-value PERC_OTHER_STATE_OR_ABROAD = 0.020 < 0.05

**❓ Holding all other regressors constant, by how much does the total number of incidents increase or decrease when the share of families with a married couple increases by 1 percentage point in the district? Please formulate a full sentence with statistical rigor!**

In [155]:
answer_how_much_increase = """
En gardant tous les autres regresseurs constants,
une augmentation d'un pourcent de la proportion de famille avec un couple marié
réduit de 2115,1839 le nombre d'incidents
"""

#### Limits of this regression model

You had asked the intern on the team to estimate a linear regression model so as to investigate the socio-economic determinants of crime in Boston. The results above are those that he presented. In the email he sent to you, he added:

> *You will probably notice the extremely high R-squared score of this model: I think we have an excellent fit and the results are solid* 😄

But you have very strong doubts about this regression and you think it is a perfect occasion to give some very important advice to your intern...

**❓  What is the main limitation of this (clearly spurious) regression according to you? This observation explains why we are getting a very high R-squared and large standard errors. Please provide your answer in the following Markdown cell.**

In [156]:
answer_limitations = """
Il y a beaucoup de variables par rapport au nombre d'observations, ce qui peut expliquer un R² élevé mais également proche de
l'overfitting ; on donne trop d'informations au modèle qui risque d'être trop adapté aux observations et donc de ne pas réussir
à généraliser pour de nouvelles données.
De plus, certaines variables ne mettent pas en évidence d'impact significatif sur le nombre d'incidents. Par exemple, 
PERC_LESS_THAN_HIGH_SCHOOL et PERC_COLLEGE_GRADUATES sont fortement non significatives (p > 0.8). En les retirant, on constate
(model2 si dessous) que les p-values des autres régresseurs diminuent et passent sous les 5% (le R² ajusté augmente).
"""

model2 = smf.ols(formula = 'NB_INCIDENTS ~ MEDIAN_AGE + PERC_OF_30_34 + PERC_MARRIED_COUPLE_FAMILY + PERC_OTHER_STATE_OR_ABROAD', data=data).fit()
model2.summary()

/home/steph/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           NB_INCIDENTS   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     15.34
Date:                Thu, 22 Dec 2022   Prob (F-statistic):            0.00142
Time:                        15:02:12   Log-Likelihood:                -114.65
No. Observations:                  12   AIC:                             239.3
Df Residuals:                       7   BIC:                             241.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   2.614e+04   2.44e+04      1.071      0.320   -3.16e+04    8.39e+04
MEDIAN_AGE                  2050.9892    715.591      2.866      0.024     358.886    3743.092
PERC_OF_30_34               -624.0065    200.407     -3.114      0.017   -1097.894    -150.119
PERC_MARRIED_COUPLE_FAMILY -2146.3222    298.220     -7.197      0.000   -2851.501   -1441.144
PERC_OTHER_STATE_OR_ABROAD  1799.0184    429.566      4.188      0.004     783.257    2814.780
==============================================================================
Omnibus:                        5.854   Durbin-Watson:                   2.283
Prob(Omnibus):                  0.054   Jarque-Bera (JB):                2.464
Skew:                           1.008   Prob(JB):                        0.292
Kurtosis:                       3.927   Cond. No.                     1.07e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.07e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 💾 Run the cell below to submit your results

In [157]:
from nbresult import ChallengeResult
import json
answers_inference = {"MEDIAN_AGE": answer_median_age,
                    "T_STAT":answer_t_statistic,
                     "P_VALUE": answer_p_value,
                     "INCREASE": answer_how_much_increase,
                     "LIMITATIONS": answer_limitations}

with open("tests/answers_inference.json", "w", encoding="utf-8") as f:
    json.dump(answers_inference, f, ensure_ascii=False, indent=4)

result = ChallengeResult(
    'linear_regression_analysis',
    model=model,
    numerator=numerator,
    denominator=denominator,
    t=t_median_age,
    pvalue=pvalue_median_age,
    regressors=significant_regressors
)
result.write()

# 2. Analysis for the police department (4h)

**This part assesses the last skill of the first block of the certification _Analyse exploratoire et descriptive de la donnée_:**
- C4 Concevoir une représentation graphique adaptée afin de visualiser les relations entre les données

**and the skills of the fourth block _Développement et Gestion de projet en IA_:**
- C18 Analyser et formuler une problématique métier du point de vue de l'analyste de données
- C19 Planification des actions à mettre en œuvre
- C20 Mobiliser les ressources nécessaires pour mener à bien le projet en respectant les délais
- C21 Recommander des actions à sa hiérarchie et rendre compte de ses résultats  

🚨 **The expected deliverable for this part is the Short presentation**  

The head of the Police Department of Boston, who read your report for the Mayor's team, was extremely interested in the results. He contacted your consulting firm for an additional presentation, that would focus on the nature of crimes that take place in Boston, the potential time trends that you could identify and/or the heterogeneity of the impact of criminality on the different police districts. 

## 2.1 Start with a fresh dataset

You will start from a fresh dataset, that corresponds more or less to the `incident_reports` table of the database.

In [28]:
# Run this cell to download the datasets in the data directory
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/incident_reports.csv > data/incident_reports.csv   
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/districts.csv > data/districts.csv    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.1M  100 19.1M    0     0   170k      0  0:01:55  0:01:55 --:--:--  164k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   498  100   498    0     0   1411      0 --:--:-- --:--:-- --:--:--  1414


In [158]:
# Load the dataset
from data import load_data_viz_data

data = load_data_viz_data()
data[['LAT','LONG']] = data[['LAT','LONG']].astype('float64')

print("Shape of the DataFrame:", data.shape)

data.head()

Shape of the DataFrame: (237221, 9)


,INCIDENT_NUMBER,OFFENSE_CODE_GROUP,SHOOTING,OCCURRED_ON_DATE,LAT,LONG,NAME,LAT_POLICE_STATION,LONG_POLICE_STATION
0,I192068249,Other,0,2015-08-28 10:20:00,42.330119,-71.084251,Roxbury,42.328894,-71.085359
1,I182074094,Violence and harassment,0,2015-09-14 09:31:00,42.315142,-71.067047,Roxbury,42.328894,-71.085359
2,I182054888,Violence and harassment,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
3,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359
4,I182054888,Other,0,2015-07-12 15:37:00,42.312243,-71.075499,Roxbury,42.328894,-71.085359


## 2.1 Further data manipulations (1h)

In this section, we are going to answer two specific questions that the head of the Police Department of Boston asked your team, about the characteristics of the incidents that occur in the various districts of the municipality.

### 2.1.1 Most represented type of incident per district

First, the head of the Police Department of Boston wants to know what incident category is most often found in each district. 

**📝 Construct a DataFrame with**
- **one row per police district, designated by its full name**
- **one text column that indicates the name of the most common category of incident in the district over the whole sample period.**

💡 Hint: you may need to first define a custom aggregation function.

In [159]:
group = data.groupby('NAME')
group = group['OFFENSE_CODE_GROUP'].agg(lambda x: x.mode())
pd.DataFrame(group)

,OFFENSE_CODE_GROUP
NAME,
Brighton,Larceny and vandalism
Charlestown,Larceny and vandalism
Dorchester,Larceny and vandalism
Downtown,Larceny and vandalism
East Boston,Larceny and vandalism
Hyde Park,Larceny and vandalism
Jamaica Plain,Larceny and vandalism
Mattapan,Police investigation procedure
Roxbury,Larceny and vandalism


**❓ Can you tell what is the second most common offense int the Brighton district?**

In [160]:
pd.DataFrame(data.groupby(by=["NAME","OFFENSE_CODE_GROUP"]).agg('count')).sort_values(["NAME", "INCIDENT_NUMBER"], ascending=[True,False]).index[1]

('Brighton', 'Police investigation procedure')

In [161]:
second_most_common_offense = "Police investigation procedure"

### Average distance to the police station per district

Second, based on the Haversine distance function defined below, the head of the Police Department would like to know, for each district, **the average distance between the location of the incident and the police station**. 

**📝 Construct a DataFrame with one row per police district, designated by its full name, and one column that displays this average Haversine distance.**

In [166]:
# Haversine distance function
from math import radians, sin, cos, asin, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Compute distance (km) between two pairs of (lat, lng) coordinates
    See - (https://en.wikipedia.org/wiki/Haversine_formula)
    """
    
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    
    return 2 * 6371 * asin(sqrt(a))

In [175]:
data['haversine'] = data.apply(lambda row: haversine_distance(row["LAT"], row["LONG"], row["LAT_POLICE_STATION"], row["LONG_POLICE_STATION"]),axis=1)

In [178]:
haversine_df = data.groupby('NAME').mean().drop(columns=["SHOOTING", "LAT", "LONG", "LAT_POLICE_STATION", "LONG_POLICE_STATION"])
haversine_df

,haversine
NAME,
Brighton,3.903264
Charlestown,57.206177
Dorchester,2.730074
Downtown,5.981918
East Boston,5.416621
Hyde Park,1.860631
Jamaica Plain,2.169588
Mattapan,1.604530
Roxbury,2.665766


**❓ Can you tell what is the average distance between the police station and the offenses in the Brighton district?**  
Store your answer as a `float` in the `average_distance_km variable`.

In [179]:
average_distance_km = 3.903264

### 💾 Run the cell below to submit your results

In [180]:
from nbresult import ChallengeResult

result = ChallengeResult('manipulation',
                         second_most_common_offense=second_most_common_offense,
                         average_distance_km=average_distance_km)
result.write()

# 2.2 Short presentation (3h)

🚨 (REQUIRED TO VALIDATE THE CERTIFICATION)

Using the latest dataset that you loaded, your mission is now to prepare 5 slides (including a title slide) that you would present to the head of the Police Department.  
You should focus on including the output of the two "Further data manipulations" tasks in your presentation.  

⚠️  You can use any presentation editor of your choice, but **the slides must be shared either in HTML or in PDF format and saved in the current directory**

Before you get started, here are four small pieces of advice:

- to prepare your data visualisations, do not hesitate to create a separate, blank notebook; 


- pay particular attention to the readability and the clarity of your legends, titles, charts and tables; 


- the Pandas `resample` method might be useful if you want to plot time trends;


- keep in mind that you are working with real data and sometimes, data are not very talkative. Do not feel discouraged if your charts do not provide revolutionary insights: typically, an absence of trend is a substantial piece of information!

Good luck in changing Boston residents' lives!

# A word of conclusion

Congratulations for going through the exercise 🎉

If you wish to pursue your analysis at some point, note that all datasets (and many others) are publicly available online, on the [Analyze Boston](https://data.boston.gov) website.

Besides, if you are interested in the topic, you can start by reading the work of Nobel Prize laureate Gary Becker, who was the first to model crime as a rational phenomenon, similarly to an economic decision. This model, although it has limitations, marked a breakthrough in the study of crime and paved the way for many empirical studies that further analysed the socio-economic determinants of illegal activities. 

👉 [Link](https://olis.leg.state.or.us/liz/2017R1/Downloads/CommitteeMeetingDocument/125036) to download a full-text version of "Crime and Punishment: An Economic Approach" by Becker (1968)